# Convert Published HSC Data Vectors to SACC

Downloaded from: http://th.nao.ac.jp/MEMBER/hamanatk/HSC16aCSTPCF/index.html

In [12]:
import numpy as np
import sacc
import pandas as pd
import h5py as h
import matplotlib.pyplot as plt

In [13]:
data_dir = '/global/cscratch1/sd/elp25/txpipe-reanalysis/hsc/data/'

In [14]:
!ls {data_dir}/

hsc16a_cstpcf_h2020.cov.txt  photometry_catalog_hsc_nonmetacal.h5
hsc16a_cstpcf_h2020.xim.txt  shear_catalog_hsc_nonmetacal.h5
hsc16a_cstpcf_h2020.xip.txt


In [15]:
#cov = pd.read_table(data_dir+'/hsc16a_cstpcf_h2020.cov.txt',names=['i','j','Cov(i,j)'],skiprows=1,delim_whitespace=True)hsc16a_cstpcf_h2020.cov.txt

In [16]:
## col-2:  xi_- (1x1)
# col-3:  xi_- (1x2)
# col-4:  xi_- (1x3)
# col-5:  xi_- (1x4)
# col-6:  xi_- (2x2)
# col-7:  xi_- (2x3)
# col-8:  xi_- (2x4)
# col-9:  xi_- (3x3)
# col-10: xi_- (3x4)
# col-11: xi_- (4x4)

In [17]:
cols = ['theta','xi11','xi12','xi13','xi14','xi22','xi23','xi24','xi33','xi34','xi44']
data_vecs_xip = pd.read_csv(data_dir+'/hsc16a_cstpcf_h2020.xim.txt',
            names=cols ,skiprows=12, delim_whitespace=True)

In [18]:
cols = ['theta','xi11','xi12','xi13','xi14','xi22','xi23','xi24','xi33','xi34','xi44']
data_vecs_xim = pd.read_csv(data_dir+'/hsc16a_cstpcf_h2020.xim.txt',
            names=cols ,skiprows=12, delim_whitespace=True)

In [19]:
!pwd

/global/cscratch1/sd/elp25/txpipe-reanalysis/hsc/notebooks


In [22]:
# From CCL tutorial, sample fake redshift distribution
z = np.linspace(0., 3., 200)
i_lim = 26. # Limiting i-band magnitude
z0 = 0.0417*i_lim - 0.744

ell = np.arange(2, 2000)
delta_ell = ell[1]-ell[0]
Ngal = 46. * 100.31 * (i_lim - 25.) # Normalisation, galaxies/arcmin^2
pz = 1./(2.*z0) * (z / z0)**2. * np.exp(-z/z0) # Redshift distribution, p(z)
dNdz = Ngal * pz # Number density distribution

## SACC

In [23]:
sacc_data = sacc.Sacc()

n_srcs = 4

# nz_lens
#dndz = data['nz_lens'].read()
#for i in range(1, n_lens+1):
#    sacc_data.add_tracer('NZ', 'lens%d' % (i-1), dndz['Z_MID'], dndz['BIN%d' % i])

#placeholder nz for now 

# nz_src
for i in range(0, n_srcs):
    print('adding tracer ',i)
    sacc_data.add_tracer('NZ', 'source_%d' % (i), z, dNdz)

# xip

for i in range(0, n_srcs):
    for j in range(i, n_srcs):
        print('adding data for combo, '+str(i)+str(j))
        theta = data_vecs_xip.theta
        xip = data_vecs_xip['xi'+str(i+1)+str(j+1)]
        
        #theta_min = min(bin_data.theta)
        #theta_max = max(bin_data.theta)

        #ij_msk = (xip['BIN1'] == i) & (xip['BIN2'] == j)
        #xip_ij = xip[ij_msk]
        #msk = (xip_ij['ANG'] > theta_min) & (xip_ij['ANG'] < theta_max)

        #tot_msk.extend(msk.tolist())

        sacc_data.add_theta_xi(
            'galaxy_shear_xi_plus',
            'source_%d' % (i),
            'source_%d' % (j),
            theta,
            xip)

for i in range(0, n_srcs):
    for j in range(i, n_srcs):
        
        theta = data_vecs_xim.theta
        xim = data_vecs_xim['xi'+str(i+1)+str(j+1)]

        sacc_data.add_theta_xi(
            'galaxy_shear_xi_minus',
            'source_%d' % (i),
            'source_%d' % (j),
            theta,
            xim)

# covmat

#sacc_data.add_covariance(cov_matrix)

sacc_data.save_fits(data_dir+'/hsc_cosmicshear_sacc_data.fits', overwrite=True)

adding tracer  0
adding tracer  1
adding tracer  2
adding tracer  3
adding data for combo, 00
adding data for combo, 01
adding data for combo, 02
adding data for combo, 03
adding data for combo, 11
adding data for combo, 12
adding data for combo, 13
adding data for combo, 22
adding data for combo, 23
adding data for combo, 33


In [21]:
!pwd

/global/cscratch1/sd/elp25/txpipe-reanalysis/hsc/notebooks
